In [5]:
import sys 
sys.path.append("/scratch/izar/kapps/DEX-Cyclic-Arbitrage/")
from config.get import cfg
import pandas as pd
import numpy as np

In [6]:
data = pd.read_csv(cfg['files']['preprocessed_data'])
data.head()

,cycle_id,token1,token2,baseAmount,quoteAmount,quotePrice,gasPrice,gasValue,time
0,0,WETH,SHUF,0.010000,10.620910,1062.091006,34.100011,0.009516,2020-05-18 17:48:23
1,0,WETH,SHUF,0.009372,10.000000,1067.003380,33.210001,0.006049,2020-05-18 17:59:48
2,0,WETH,SHUF,0.010000,10.592990,1059.298991,33.100009,0.005471,2020-05-18 18:01:30
3,0,WETH,SHUF,0.010000,10.143472,1014.347243,33.100009,0.007748,2020-05-18 18:04:30
4,0,WETH,SHUF,0.009804,10.000000,1020.026068,33.100009,0.006029,2020-05-18 18:04:54


In [7]:
# from data exploration log was found to be a good normaliser
def log_scaling(data):
    data['baseAmount']  = np.log(data.baseAmount)
    data['quoteAmount'] = np.log(data.quoteAmount)
    data['quotePrice']  = np.log(data.quotePrice)
    return data;

log_data = log_scaling(data)

In [8]:
N_TOKEN = 3 # cycle length
K = 2       # quote price & gasPrice
N = 1000    # number of cycles
P = 600     # max time series length per cycle

def build_tensor(data):
    tensor = np.zeros((N, N_TOKEN,P, K))
    i = 0
    def get_sorted_token_map(g):
        t = g[['token1','token2']].values
        u, ind = np.unique(t, return_index=True)
        u_sorted =  u[np.argsort(ind)]
        return dict(zip(u_sorted, range(len(u_sorted))))

    for _, group in iter(data.groupby('cycle_id')):
        token_map = get_sorted_token_map(group)
        for _, g in iter(group.groupby(['token1','token2'])):
            a = g[['quotePrice','gasPrice']].values
            # zero padding
            padded = np.pad(a, [(0, P - len(a)),(0,0)])
            # assign and reshape into a matrix
            first_token = g.token1.iloc[0]
            token_ind = token_map[first_token]
            tensor[i,token_ind,:,:] = padded.reshape(1,P,K)
        i = i+1
    return tensor
    
X = build_tensor(log_data)

In [9]:
print("data shape : ",X.shape)

data shape :  (1000, 3, 600, 2)


In [12]:
np.save(cfg['files']['raw_features'],X)

In [13]:
X = np.load(cfg['files']['raw_features'])

In [14]:
X.shape

(10000, 3, 600, 2)